In [ ]:
import random

In [ ]:
simulazioni = 100000
n_d8 = 11
threshold = 74

for livello in range(9):
    
    n_d8 = 5 + 2*livello
    distribuzione_res = [[], [], [], []]
    labels = ['normale', 'valore minimo dado 4', 'se < 4 ritira dado']
    for _ in range(simulazioni):
        sums = [0, 0, 0, 0] # standard, min 4, se  < 4 ritira, numero ritiri
        for _ in range(n_d8):
            dado = random.randint(1, 8)
            sums[3] += 1
            sums[0] += dado
            sums[1] += max(dado, 4)
            while dado < 4:
                dado = random.randint(1, 8)
                sums[3] += 1
            sums[2] += dado
        distribuzione_res[0].append(sums[0] >= threshold)
        distribuzione_res[1].append(sums[1] >= threshold)
        distribuzione_res[2].append(sums[2] >= threshold)
        distribuzione_res[3].append(sums[3])
    
    print(f'Probabilità per slot livello {livello + 1} ({n_d8}) d8')
    for i, d in enumerate(distribuzione_res):
        if i > 2:
            print(f'    numero di tiri medio: {sum(distribuzione_res[3])/simulazioni}')
            break
        print(f'  {labels[i]}')
        print(f'    risultato >= {threshold}: {(100*sum(d))/simulazioni}%')
        
    
    
    
    

Probabilità per slot livello 1 (5) d8
  normale
    risultato >= 74: 0.0%
  valore minimo dado 4
    risultato >= 74: 0.0%
  se < 4 ritira dado
    risultato >= 74: 0.0%
    numero di tiri medio: 8.00136
Probabilità per slot livello 2 (7) d8
  normale
    risultato >= 74: 0.0%
  valore minimo dado 4
    risultato >= 74: 0.0%
  se < 4 ritira dado
    risultato >= 74: 0.0%
    numero di tiri medio: 11.20108
Probabilità per slot livello 3 (9) d8
  normale
    risultato >= 74: 0.0%
  valore minimo dado 4
    risultato >= 74: 0.0%
  se < 4 ritira dado
    risultato >= 74: 0.0%
    numero di tiri medio: 14.39305
Probabilità per slot livello 4 (11) d8
  normale
    risultato >= 74: 0.06%
  valore minimo dado 4
    risultato >= 74: 0.14%
  se < 4 ritira dado
    risultato >= 74: 5.614%
    numero di tiri medio: 17.59118
Probabilità per slot livello 5 (13) d8
  normale
    risultato >= 74: 3.409%
  valore minimo dado 4
    risultato >= 74: 16.41%
  se < 4 ritira dado
    risultato >= 74: 80.898

In [21]:
simulazioni = 1000000
threshold = 14

livello = 3
n_d8 = 5 + 2*livello
distribuzione_res = []
labels = ['normale', 'valore minimo dado 4', 'se < 4 ritira dado']
for _ in range(simulazioni):
    sums = 0 # standard, min 4, se  < 4 ritira, numero ritiri
    for _ in range(n_d8):
        dado = random.randint(1, 8)
        sums += 1
        while dado < 4:
            dado = random.randint(1, 8)
            sums += 1
    distribuzione_res.append(sums <= threshold)

print(f'Probabilità per slot livello {livello + 1} ({n_d8}) d8')
print(f'  risultato <= {threshold}: {(100*sum(distribuzione_res))/simulazioni}%')
    

    
    
    

Probabilità per slot livello 4 (11) d8
  risultato <= 14: 16.8623%


In [9]:
m1 = "WR NOIVBIG LVJCFW JCL NFOWMZZL XSZOMUFLVW OOTO GSQBO GQ BCFBOF TCSGWI GSUI"
m2 = "BHTZS FIHOFWAPH LOJDVHW DTZO VIBHD ZSQLBO ZD KVWDDS"

In [10]:
def vigenere_decrypt(ciphertext, key):
    """
    Decifra un testo cifrato con il cifrario di Vigenère mantenendo spazi e punteggiatura.
    
    Args:
        ciphertext (str): testo cifrato
        key (str): chiave per la decifratura
    
    Returns:
        str: testo in chiaro
    """
    key = key.upper()
    plaintext = []
    key_index = 0

    for c in ciphertext:
        if c.isalpha():
            c_upper = c.upper()
            c_index = ord(c_upper) - ord('A')
            k_index = ord(key[key_index % len(key)]) - ord('A')
            p_index = (c_index - k_index) % 26
            p_char = chr(p_index + ord('A'))

            # Mantieni maiuscola/minuscola originale
            if c.islower():
                p_char = p_char.lower()
            plaintext.append(p_char)

            key_index += 1
        else:
            # Mantieni spazi e punteggiatura
            plaintext.append(c)

    return ''.join(plaintext)


In [11]:
vigenere_decrypt(m1, "ODIO")

'IO FAUSTUS INVOCO VOI FRATELLI PELLEGRINI ALLA SPINA DI NOCTAR QUESTA SERA'

In [12]:
vigenere_decrypt(m2, "ODIO")

'NELLE CATACOMBE DAVANTI ALLA SANTA RECITA LA CHIAVE'

In [ ]:
import json



[1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243]


In [24]:
from pathlib import Path

In [26]:
for path in Path('./results/final_check/').iterdir():
    with open('./results/final_check/4314740.json', 'r') as f:
        a = json.loads(f.read())
    
    seeds = []
    for el in a['data']:
        seeds.append(el[1]['seed'])
    print(sorted(seeds))

[1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243]
[1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243]
[1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243]
[1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243]
[1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243]
[1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243]


In [8]:
a['data'][0][1]

{'id': 8,
 'success': True,
 'collisions': 0,
 'reward': 13.721108436584473,
 'length': 401,
 'seed': 1234,
 'distance_traveled': 6.804683208465576,
 'global_avg_dist_obstacle': 2.367732286453247,
 'global_avg_visibility': 6.436572074890137,
 'global_avg_dispersion': 4.796193599700928,
 'global_characteristic_dimension': 6.733709335327148,
 'path_tortuosity': 1.0186641216278076,
 'path_length': 22.818077087402344}